<a href="https://colab.research.google.com/github/sdhilip200/Twitter-Scraping-Sentiment-Analysis/blob/main/Twitter_Scraping_and_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import itertools
import snscrape.modules.twitter as sntwitter
import plotly.graph_objects as go
from datetime import datetime

In [6]:
start_time = datetime.now()
limit = 1000
#Creating dataframe called 'data' and storing the tweets from May 1st 2021 to 30th Juy 2021 for 'Vaccine'
data = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('"#vaccine since:2021-05-01 until:2021-07-30"').get_items(),limit))

end_time = datetime.now()

# Printing the time taken to scrape these tweets
print('Duuration: {} '.format(end_time - start_time))


Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%22%23vaccine+since%3A2021-05-01+until%3A2021-07-30%22&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel: non-200 status code
4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%22%23vaccine+since%3A2021-05-01+until%3A2021-07-30%22&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel failed, giving up.

In [ ]:
#Checking the columns
data.columns

Index(['url', 'date', 'content', 'renderedContent', 'id', 'user', 'replyCount',
       'retweetCount', 'likeCount', 'quoteCount', 'conversationId', 'lang',
       'source', 'sourceUrl', 'sourceLabel', 'outlinks', 'tcooutlinks',
       'media', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId',
       'inReplyToUser', 'mentionedUsers', 'coordinates', 'place', 'hashtags',
       'cashtags'],
      dtype='object')

In [ ]:
data.to_csv("tweet.csv")

In [ ]:
#keeping only date, id, content, user, and hashtag and stored into dataframe called 'data'
df = data[['date', 'id', 'user', 'content']]


df1 = df.head(5000)

In [ ]:
# Sentiment Analysis

#Sentiment Analysis

In [ ]:
#Importing pipeline from Transformers

from transformers import pipeline
sentiment_classifier = pipeline(model='cardiffnlp/twitter-roberta-base-sentiment-latest"')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
# Passing the tweets into the sentiment pipeline and extracting the sentiment score and label
df1 = (
    df1
    .assign(sentiment = lambda x: x['content'].apply(lambda s: sentiment_classifier(s)))
    .assign(
         label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
         score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
    )
)

In [ ]:
#df1.head(20)
df1.head(200).to_csv("tweets.csv", index=False)

In [ ]:
df1['content'][0]

'Keep follow me all LunaC dreamer #Terra_Luna #TerraClassic #lunaburn #LUNAClassicCommunity #lUNACBURN'

In [ ]:
df1['content'][10]

'TERRA LUNA SUBIU QUASE 200% OQUE ESTÁ ACONTECENDO? ANONYMOUS AMEAÇA CEO DA TERRA LUNA #terraluna \n\nAssista: https://t.co/NP2HNONcRe'

In [ ]:
df1['content'][14]

'2 gündür durmuyor. Sıradan bir coin olsa pump akabinde dump derim. Ama öyle olmuyor kademeli şekilde artıyor/arttırılıyor. Merak ettiğim neden artıyor. 1$ olması kime ne kazanç sağlar. Kafamda çok soru var ama anladığım, çözen yok.\n#ustc #Terra #binance #LUNA #LUNC #Terra_Luna https://t.co/EWBQk1VxiO'

In [ ]:
#Visualizing the sentiments


fig = go.Figure()

fig.add_trace(go.Bar(x = df1["score"],
                    y = df1["label"],
                orientation = "h")) #set orientation to horizontal because we want to flip the x and y-axis

fig.update_layout(plot_bgcolor = "white")              
                                #apply our custom category order

fig.show()